In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import gc


In [ ]:
import pickle

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
import cv2

In [ ]:

import torchvision.models.detection as tmd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor as FRP
import inspect
import torchvision
#converting the customised rcnn model
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


In [ ]:
root_dir = 'D:\\Drone-Object-Detection\\VisDroneDataset\\'
test_dir = 'test\\'
train_dir = 'train\\'
val_dir = 'validation\\'

In [ ]:
def split_image_with_asp_ratio(image):
    print(image.shape)

In [ ]:
class VisDroneDataset(Dataset):
    def __init__(self, path, img_shape):
        
        self.root = path
        self.width = img_shape[1]
        self.height = img_shape[0]
        self.images = os.listdir(path+'images\\')
        self.annotations = os.listdir(path+'annotations\\')
        self.length = len(self.images)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        #print(self.root+'images\\'+self.images[idx])
        image = cv2.imread(self.root+'images\\'+self.images[idx])
        #print(image.shape)
        img_width = image.shape[1]
        img_height = image.shape[0]     #right?
        
        image  =cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        if ((self.width,self.height) != (-1,-1)):
            image = cv2.resize(image,(self.width,self.height),interpolation = cv2.INTER_AREA)
        else:
            self.width = img_width
            self.height = img_height
            
        image = image.transpose(2,0,1)
        image = (image/255.0)
        
        
        #bbox_left, bbox_top, bbox_width, bbox_height, category = map(float, data[:5])
        target = self.annotations[idx]    #*.txt file       
        
        boxes = []
        labels = []
        with open(self.root+'annotations\\'+target) as f:
            for line in f:
                data = line.split(',')#strip
                #print(list(map(float, data[:5])))
                box_left, box_top, box_width, box_height,smth, category = map(float, data[:6])
                
                box_width = 1 if box_width < 1 else box_width
                box_height = 1 if box_height < 1 else box_height
                
                box_right = box_left + box_width
                box_bottom = box_top + box_height
                #rescale box            
                box_left = (box_left/img_width) * self.width
                box_right = (box_right/img_width) * self.width
                box_top = (box_top/img_height) * self.height
                box_bottom = (box_bottom/img_height) * self.height
                boxes.append((box_left,box_top,box_right, box_bottom))
#                 for b in boxes:
#                     for bb in b:
#                         if bb > 480 or bb < 0:
#                             print(b,'\n',bb)
                labels.append(int(category))
                
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #print(boxes)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor((idx))
        
        return image, target

In [ ]:
train_dataset = VisDroneDataset(root_dir+train_dir,(480,480))
val_dataset = VisDroneDataset(root_dir+val_dir,(-1,-1))

In [ ]:
def collate_fn(batch):
    images = []
    targets = []

    for img, target in batch:
        images.append(img)
        targets.append(target)

    return (images, targets)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=3, shuffle=True,collate_fn=collate_fn)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.hub.set_dir('D:\\torch_models')

In [ ]:
from torchvision.models import resnet50

In [ ]:
from torchvision.models.detection import ssdlite320_mobilenet_v3_large

def create_model_ssd():
    model = ssdlite320_mobilenet_v3_large(num_classes=12)
    print(model)
    return model


def create_model_faster_rcnn(num_classes):
    # Load a pre-trained  R-CNN model 
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    for p in model.parameters():
        p.requires_grad=False
        
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model



model = create_model_faster_rcnn(12).to(device)

model.train()

import time
from tqdm import tqdm
def train_one_epoch(model, train_loader, optimizer):
    total_loss=0
    n=0
    prev_time = time.time()
    for data, target in tqdm(train_loader):
        optimizer.zero_grad()
        data = torch.as_tensor(np.array(data),dtype=torch.float32).to(device)
        target = [{k: v.to(device) for k, v in t.items()} for t in target]
        loss_dict = model(data, target)

        losses = sum(loss for loss in loss_dict.values())

        total_loss+=losses.item()
        losses.backward()
        optimizer.step()
        if(n % 20 == 0):
            print('loss', total_loss/(n+1))
            prev_time = time.time()
        n+=1
        del data, target
        #model = model.to(torch.device('cpu'))
        gc.collect()
        torch.cuda.empty_cache()
        
        #model = model.to(device)
    
    print(total_loss/n,'a')

num_epoch = 5
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=0.0003)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
model = model.to(device)


print(len(params))
print(len(list(model.parameters())))


for epoch in range(num_epoch):
    train_one_epoch(model,train_loader,optimizer)
    lr_scheduler.step()

In [ ]:
model = None

In [ ]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
def choose_model(which):
    model = None
    if which == 'cropped':
        with open('model_trained_on_cropped.pkl', 'rb') as f:
            model = pickle.load(f)
            return model
    else:
        with open('model.pkl', 'rb') as f:
            model = pickle.load(f)
            return model

In [ ]:
model.eval()

In [ ]:

gc.collect()

torch.cuda.empty_cache()

from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    #cuda.select_device(0)
    #cuda.close()
    #cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()    

In [ ]:
def box_iou_batch(boxes_a: np.ndarray, boxes_b: np.ndarray) -> np.ndarray:

    def box_area(box):
        return (box[2] - box[0]) * (box[3] - box[1])

    area_a = box_area(boxes_a.T)
    area_b = box_area(boxes_b.T)

    top_left = np.maximum(boxes_a[:, None, :2], boxes_b[:, :2])
    bottom_right = np.minimum(boxes_a[:, None, 2:], boxes_b[:, 2:])

    area_inter = np.prod(
    	np.clip(bottom_right - top_left, a_min=0, a_max=None), 2)
        
    return area_inter / (area_a[:, None] + area_b - area_inter)

In [ ]:
def non_max_suppression(predictions, iou_threshold: float = 0.9) -> np.ndarray:
    #print(predictions)
    predictions = torch.cat((predictions['boxes'] , predictions['scores'].unsqueeze(dim=1),\
                             predictions['labels'].unsqueeze(dim=1)),dim=1) .detach().cpu().numpy()
    
    
    sort_index = np.flip(predictions[:, 4].argsort())
    
    rows,columns = predictions.shape
    
    predictions = predictions[sort_index]

    boxes = predictions[:, :4]
    categories = predictions[:, 5]
    ious = box_iou_batch(boxes, boxes)
    ious = ious - np.eye(rows)

    keep = np.ones(rows, dtype=bool)

    for index, (iou, category) in enumerate(zip(ious, categories)):
        if not keep[index]:
            continue

        condition = (iou > iou_threshold) & (categories == category)
        keep = keep & ~condition

    return predictions[keep[sort_index.argsort()]]

In [ ]:
model.eval()


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.patches as patches

CONFIDENCE_THRESHOLD = 0.3

In [ ]:
labels_map = ['Background', 'Pedestrian', 'People', 'Bycicle', 'Car', 'Van', 'Truck',
                'Tricycle', 'Awning-tricycle', 'Bus', 'Motor', 'Others']
def draw_bounding_boxes(img, boxes,labels,scores, ax):   # image - tensor, others - numpy
    
    ax.imshow(img.detach().cpu().numpy().transpose(1,2,0))
    ax.set_title('title')
    threshold = CONFIDENCE_THRESHOLD
    for box,label,score in zip(boxes,labels.astype(np.uint8),scores):
        #print(label)
        if(score < threshold):
            continue
        x1, y1, x2, y2 = box
        width = x2 - x1
        height = y2 - y1
        rect = patches.Rectangle((x1, y1), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(
                x1,
                y1,
                f"{labels_map[label]}: {(np.round(score*100,1))}%",
                verticalalignment='top',
                color='white',
                fontsize=8,
                bbox={'facecolor': 'red', 'alpha': 0.7, 'pad': 1}
            )
    #plt.show()
    return ax

In [ ]:
from torchvision.ops import nms

def non_max_suppression_git(preds, iou_threshold=0.7, score_threshold=0.3):
    
    boxes = preds['boxes']
    scores = preds['scores']
    labels = preds['labels']

    # Filter out predictions below the score_threshold
    keep_idx = scores > score_threshold
    boxes = boxes[keep_idx]
    scores = scores[keep_idx]
    labels = labels[keep_idx]

    # Perform NMS and get the indices of the remaining predictions
    keep_indices = nms(boxes, scores, iou_threshold)

    # Filter out the predictions using the keep_indices
    nms_boxes = boxes[keep_indices]
    nms_scores = scores[keep_indices]
    nms_labels = labels[keep_indices]

    # Create a dictionary containing the filtered predictions
    nms_preds = {
        'boxes': nms_boxes,
        'scores': nms_scores,
        'labels': nms_labels
    }

    return nms_preds


image = cv2.imread('D:\\VisDroneDataset\\test\\images\\0000009_01339_d_0000005.jpg')
image  =cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
image = cv2.resize(image,(480,480),interpolation = cv2.INTER_AREA)
image = image.transpose(2,0,1)

image = torch.as_tensor(image, dtype=torch.float32).unsqueeze(dim=0).to(device)

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
def draw_predictions(imgs, preds, mode, i):#preds - list of dictionaries of tensors
    fig, ax = plt.subplots(1,len(imgs), figsize=(40,40))
    for idx,(img,ann) in enumerate(zip(imgs, preds)):
        #print(ann)
        annotations_new = []
        annotations_new.append(torch.cat((ann['boxes'] ,\
                                           (ann['scores']).unsqueeze(dim=1) if mode=='pred' else torch.ones(len(ann['boxes'])).\
                                          unsqueeze(dim=1),\
                                           ann['labels'].unsqueeze(dim=1)),dim=1).detach().cpu().numpy())
        ann = np.array(annotations_new[0])
        #print(ann)
        ann = ann[ann[:,4] > CONFIDENCE_THRESHOLD,:]
        #print(ann)
        
        draw_bounding_boxes(img,ann[:,0:4],ann[:,5],ann[:,4],ax[idx] if len(imgs) != 1 else ax)
        
    if(i!=-1):
        fig.savefig(f'D:\\Drone-Object-Detection\\Results\\Images\\img{i}.png')
        
    plt.show()

In [ ]:
from torchmetrics.detection import MeanAveragePrecision

In [ ]:
def replace_cuda_with_cpu(data):
    if isinstance(data, torch.Tensor):
        return data.cpu()
    elif isinstance(data, dict):
        return {key: replace_cuda_with_cpu(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [replace_cuda_with_cpu(item) for item in data]
    else:
        return data

In [ ]:
def dict_to_cpu(preds):
    new_l = []
    for dic in preds:
        new_l.append(replace_cuda_with_cpu(dic))
        
    #print(new_l)
    return new_l


In [ ]:
def from_dict_to_arr(dictionary, data = 'target'):
    annotations_new = []
    dictionary = dict_to_cpu(dictionary)
    
    for ann in dictionary:
        
        annotations_new+=(torch.cat((ann['boxes'] ,\
                                           torch.ones(len(ann['boxes'])).unsqueeze(dim=1) if data == 'target' else \
                                     ann['scores'].unsqueeze(dim=1),\
                                           ann['labels'].unsqueeze(dim=1)),dim=1))
    
    
    return annotations_new

In [ ]:
def calc_mAP(preds,target):
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    
    return metric.compute()


In [ ]:
def make_all_numpy(obj):
    if isinstance(obj, torch.Tensor):
        return obj.numpy()
    elif isinstance(obj, (list, tuple)):
        return [make_all_numpy(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: make_all_numpy(value) for key, value in obj.items()}
    else:
        return obj

In [ ]:
#detections (Array[N, 6]), x1, y1, x2, y2, conf, class
# labels (Array[M, 5]), class, x1, y1, x2, y2
def prepare_for_conf_matrx(pred,annot):
    pred = from_dict_to_arr(dict_to_cpu(pred),'my_pred')
    annot = from_dict_to_arr(dict_to_cpu(annot),'target')
    
    pred = make_all_numpy(pred)
    annot = make_all_numpy(annot)
    
    permutation = [5,0,1,2,3,4]
    
    pred, annot = np.array(pred), np.delete(np.array(annot,dtype=np.int16)[:,permutation] , 5,1)
    #print(pred)
    pred[:,0] = pred[:,0].astype(np.int16)
    pred[:,1] = pred[:,1].astype(np.int16)
    pred[:,2] = pred[:,2].astype(np.int16)
    pred[:,3] = pred[:,3].astype(np.int16)
    pred[:,5] = pred[:,5].astype(np.int16)
    #pred[:,[0,1,2,3,5]] = np.array(pred[:,[0,1,2,3,5]],dtype=np.uint8)
    
    #print(pred)
    return pred,annot

In [ ]:

from sklearn.metrics import confusion_matrix

def build_confusion_matrix(pred_boxes, gt_boxes):
    
    pred_labels = pred_boxes[:, -1]
    gt_labels = gt_boxes[:, 0]

    
    pred_labels = pred_labels.astype(np.int16)
    gt_labels = gt_labels.astype(np.int16)

    
    conf_matrix = confusion_matrix(gt_labels, pred_labels)

    return conf_matrix


In [ ]:
import seaborn as sns
%matplotlib inline

from torchvision import ops

In [ ]:
import numpy as np
def calculate_iou_matrix(boxesA, boxesB):

    # Convert numpy arrays to torch tensors
    boxesA_torch = torch.tensor(boxesA, dtype=torch.float32)
    boxesB_torch = torch.tensor(boxesB, dtype=torch.float32)

    # Use torchvision.ops.box_iou to calculate IoU matrix
    iou_matrix_torch = torchvision.ops.box_iou(boxesA_torch, boxesB_torch)

    # Convert the torch tensor back to a numpy array
    iou_matrix = iou_matrix_torch.numpy()

    return iou_matrix



In [ ]:


class ConfusionMatrix:
    def __init__(self, num_classes: int, CONF_THRESHOLD=0.3, IOU_THRESHOLD=0.5):
        self.matrix = np.zeros((num_classes + 1, num_classes + 1))
        self.num_classes = num_classes
        self.CONF_THRESHOLD = CONF_THRESHOLD
        self.IOU_THRESHOLD = IOU_THRESHOLD

    def process_batch(self, detections, labels: np.ndarray):
        
        
        gt_classes = labels[:, 0].astype(np.uint8)
        try:
            detections = detections[detections[:, 4] > self.CONF_THRESHOLD]
        except IndexError or TypeError:
            # detections are empty, end of process
            for i, label in enumerate(labels):
                gt_class = gt_classes[i]
                self.matrix[self.num_classes, gt_class] += 1
            return

        detection_classes = detections[:, 5].astype(np.uint8)
        #print(labels[:, 1:])
        #print()
        #print(detections[:, :4])
        all_ious = calculate_iou_matrix(labels[:, 1:], detections[:, :4])
        #print(detection_classes)
        want_idx = np.where(all_ious > self.IOU_THRESHOLD)
        #print(want_idx)
        all_matches = [[want_idx[0][i], want_idx[1][i], all_ious[want_idx[0][i], want_idx[1][i]]]
                       for i in range(want_idx[0].shape[0])]

        all_matches = np.array(all_matches)
        
        #print(all_matches)
        if all_matches.shape[0] > 0:  # if there is match
            all_matches = all_matches[all_matches[:, 2].argsort()[::-1]]
            all_matches = all_matches[np.unique(all_matches[:, 1], return_index=True)[1]]
            all_matches = all_matches[all_matches[:, 2].argsort()[::-1]]
            all_matches = all_matches[np.unique(all_matches[:, 0], return_index=True)[1]]

            
        #print(all_matches)
        #print(len(all_matches))
        for i, label in enumerate(labels):
            gt_class = gt_classes[i]
            
            if all_matches.shape[0] > 0 and all_matches[all_matches[:, 0] == i].shape[0] == 1:#==1?
                
                detection_class = detection_classes[int(all_matches[all_matches[:, 0] == i, 1][0])]
                self.matrix[detection_class, gt_class] += 1
            else:
                
                self.matrix[self.num_classes, gt_class] += 1

        for i, detection in enumerate(detections):
            if not all_matches.shape[0] or ( all_matches.shape[0] and all_matches[all_matches[:, 1] == i].shape[0] == 0 ):
                detection_class = detection_classes[i]
                self.matrix[detection_class, self.num_classes] += 1

    def return_matrix(self):
        return self.matrix

    def print_matrix(self):
        for i in range(self.num_classes + 1):
            print(' '.join(map(str, self.matrix[i])))
            
            
#SMTH wrong woth all matches

In [ ]:
import sys
import numpy

In [ ]:
def draw_heatmap(conf_matr, ax, cen=None):
    
    hm = sns.heatmap(np.array(conf_matr.return_matrix()), center =cen,cmap="crest",  annot=True, ax=ax,fmt='g')
    hm.set_ylabel('Predicted')
    hm.set_xlabel('True')
    hm.set_ylabel('Predicted')
    hm.set_xlabel('True')


    my_lm = labels_map.copy()
    my_lm += ['None']
    tickvalues = range(0,len(my_lm))
    hm.set_xticks(ticks=tickvalues, labels=my_lm, rotation = 'vertical')
    hm.set_yticks(ticks=tickvalues, labels=my_lm, rotation = 'horizontal')
    return hm
    
    


In [ ]:
def count_avg_map_and_conf_matr(map_file, loader,  value_to_break = 20):
    conf_matr = ConfusionMatrix(12,CONF_THRESHOLD = CONFIDENCE_THRESHOLD, IOU_THRESHOLD = 0.5)
    
    c = 0
    total_map = 0
    total_map_50 = 0
    total_map_75 = 0
    total_map_small = 0
    total_map_medium = 0
    total_map_large = 0
    
    c_total_map_50 = 0
    c_total_map_75 = 0
    c_total_map_small = 0
    c_total_map_medium = 0
    c_total_map_large = 0
    
    total_preds=0
    for i, (batch_imgs, annotations) in enumerate(loader):
        batch_imgs = [torch.from_numpy(img).float().to(device) for img in batch_imgs]#tensor
        with torch.no_grad():
            preds = model(batch_imgs)#tensor
        preds = replace_cuda_with_cpu(preds) 

        nms_preds = []
        for pred in preds:
            nms_pred = non_max_suppression_git(pred)
            nms_preds.append(nms_pred)
        #print(nms_preds)
        map_dict = calc_mAP(nms_preds, annotations)
       
        if(map_dict['map'].item() > 0):
            total_map += map_dict['map'].item()
            c+=1
            
        if(map_dict['map_50'].item() > 0):
            total_map_50 += map_dict['map_50'].item()
            c_total_map_50+=1
            
        if(map_dict['map_75'].item() > 0):
            total_map_75 += map_dict['map_75'].item()
            c_total_map_75+=1
            
        if(map_dict['map_small'].item() > 0):
            total_map_small += map_dict['map_small'].item()
            c_total_map_small+=1
            
        if(map_dict['map_medium'].item() > 0):
            total_map_medium += map_dict['map_medium'].item()
            c_total_map_medium+=1
            
        if(map_dict['map_large'].item() > 0):
            total_map_large += map_dict['map_large'].item()
            c_total_map_large+=1
        
        
        for pred in nms_preds:
            preds_ind = pred['scores'] > CONFIDENCE_THRESHOLD
            pred = pred['labels'][preds_ind]
            total_preds+= sum(pred == 5)

        
            print(c)
        a,b = prepare_for_conf_matrx(nms_preds,annotations)


        conf_matr.process_batch(a,b)


        if(i == value_to_break):
            break

            
    
    #print(total_preds)
    #map_file.write('avarage map:',total_map / c)
    
    maps = ""
    maps+= ('avgerage map 50-95: '+str(total_map/c) + '\n' + \
            'avgerage map 50: '+str(total_map_50/c_total_map_50) + '\n' + \
            'avgerage map 75: '+str(total_map_75/c_total_map_75) + '\n' + \
            'avgerage map small: '+str(total_map_small/c_total_map_small) + '\n' + \
            'avgerage map medium: '+str(total_map_medium/c_total_map_medium) + '\n' + \
            'avgerage map large: '+str(total_map_large/c_total_map_large if c_total_map_large != 0 else -1) + '\n')
            
            
    
    
    fig, ax = plt.subplots(figsize=(10,10))
    
    hm = draw_heatmap(conf_matr,ax)
    
    print(maps)
    
    map_file.write(maps)
    fig.savefig(f'D:\\Drone-Object-Detection\\Results\\conf_matrix.png')
    #plt.show()

#ОШИБКА В ТИПАХ БЫЛА(np.uint8 неправильно)

In [ ]:
model = choose_model('not').to(device)
model.eval()


In [ ]:
map_file = open('Results\\mAPs\\file.txt','w')
map_file.close()
map_file = open('Results\\mAPs\\file.txt','a')
model = model.to(device)

In [ ]:
val_loader_for_map = DataLoader(val_dataset, batch_size=1, shuffle=True,collate_fn=collate_fn)

In [ ]:
count_avg_map_and_conf_matr(map_file=map_file, loader=val_loader_for_map, value_to_break=100) # for a random data from loader dataset

In [ ]:
map_file.close()

In [ ]:
def draw_3_preds():
    
    i1 = val_dataset.__getitem__(54)
    i2 = val_dataset.__getitem__(315)
    i3 = val_dataset.__getitem__(376)

    batch_imgs = np.stack((i1[0],i2[0],i3[0]))

    l=[]
    l.append(i1[1])
    l.append(i2[1])
    l.append(i3[1])

    annotations = l
    
    
    batch_imgs = [torch.from_numpy(img).float().to(device) for img in batch_imgs]#tensor
    with torch.no_grad():
        preds = model(batch_imgs)#tensor
    preds = replace_cuda_with_cpu(preds) 
    print(preds)



    nms_preds = []
    for pred in preds:
        nms_pred = non_max_suppression_git(pred)
        nms_preds.append(nms_pred)

    draw_predictions(batch_imgs,nms_preds,'pred',1)#list of dictionaries of tensors
    draw_predictions(batch_imgs,annotations,'tar',-1)


    #НЕ МЭТЧИТ!!!

In [ ]:
draw_3_preds()